In [6]:
pip install netron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnx
import onnxruntime as rt
from torchviz import make_dot
import netron
from modeci_mdf.interfaces.pytorch import pytorch_to_mdf
import torch
import torchvision.models as models


In [8]:
model1 = models.resnet18()
model1.load_state_dict(torch.load('/content/resnet18.pth'))


<All keys matched successfully>

In [9]:
def main():
    # changed import call
    from modeci_mdf.execution_engine import EvaluableGraph

    # Create some test inputs for the model
    input_images = torch.zeros((1, 3, 224, 224), requires_grad=False)

    # Seed the random number generator to get deterministic behavior for weight initialization
    torch.manual_seed(0)

    model = model1

    model.eval()
    # Run the model once to get some ground truth outpot (from PyTorch)
    output = model(input_images)

    from modelspec.utils import _val_info

    print("Evaluated the graph in PyTorch, output: %s" % (_val_info(output)))

    # Convert to MDF
    mdf_model, params_dict = pytorch_to_mdf(
        model=model,
        args=(input_images),
        trace=True,
    )

    # Get the graph
    mdf_graph = mdf_model.graphs[0]

    # Add inputs to the parameters dict so we can feed this to the EvaluableGraph for initialization of graph input.
    params_dict["input1"] = input_images.numpy()

    # Evaluate the model via the MDF scheduler
    eg = EvaluableGraph(graph=mdf_graph, verbose=False)
    eg.evaluate(initializer=params_dict)
    output_mdf = eg.output_enodes[0].get_output()

    print("Evaluated the graph in PyTorch, output: %s" % (_val_info(output_mdf)))
    # Make sure the results are the same between PyTorch and MDF
    assert np.allclose(
        output.detach().numpy(),
        output_mdf,
    )
    print("Passed all comparison tests!")

    # Output the model to JSON
    mdf_model.to_json_file("sample1_pytorch_to_mdf.json")

    import sys

    # Exporting as onnx model
    torch.onnx.export(
        model,
        input_images,
        "sample1_pytorch_to_mdf.onnx",
        verbose=True,
        input_names=[],
        opset_version=9,
    )
    onnx_model = onnx.load("sample1_pytorch_to_mdf.onnx")
    onnx.checker.check_model(onnx_model)
    sess = rt.InferenceSession("sample1_pytorch_to_mdf.onnx")
    res = sess.run(None, {sess.get_inputs()[0].name: input_images.numpy()})
    print("Exported to MDF and ONNX")

    # export to mdf graph
    if "-graph" in sys.argv:
        mdf_model.to_graph_image(
            engine="dot",
            output_format="png",
            view_on_render=False,
            level=1,
            filename_root="sample1_pytorch_to_mdf.1",
            only_warn_on_fail=True,  # Makes sure test of this doesn't fail on Windows on GitHub Actions
            is_horizontal=True,
            rankdir="TD"
          
        )
        mdf_model.to_graph_image(
            engine="dot",
            output_format="png",
            view_on_render=False,
            level=3,
            filename_root="sample1_pytorch_to_mdf",
            only_warn_on_fail=True,  # Makes sure test of this doesn't fail on Windows on GitHub Actions
          
        )
    # export to PyTorch graph
    if "-graph-torch" in sys.argv:
        make_dot(output, params=dict(list(model.named_parameters()))).render(
            "sample1_pytorch_to_mdf_torchviz", format="png"
        )
    # export to onnx graph
    if "-graph-onnx" in sys.argv:
        netron.start("sample1_pytorch_to_mdf.onnx")


if __name__ == "__main__":
    main()



Evaluated the graph in PyTorch, output: tensor([[-2.8774e-02, -5.2932e-03, -3.6097e-02,  6.8579e-03, -3.4067e-03,
         -3.0889e-02,  2.4735e-02,  4.2477e-02,  2.6443e-03,  2.1936e-02,
         -3.2019e-02,  3.1720e-02, -4.2142e-02, -2.9292e-02,  1.8174e-02,
         -1.4973e-02,  2.7067e-02, -9.0325e-03,  2.1031e-02, -9.1640e-03,
         -6.0370e-03, -1.6253e-02, -4.0052e-02,  3.8943e-02,  3.0394e-02,
         -1.6035e-02,  3.3327e-02,  1.1011e-02,  2.5651e-02,  6.3637e-03,
         -4.0436e-02, -3.2286e-02, -5.7262e-03, -3.6450e-02, -3.5967e-02,
         -2.9956e-02, -3.4702e-04,  1.7811e-02,  5.9253e-03,  3.3936e-03,
          1.7460e-02,  4.2890e-02,  3.0843e-02,  4.2885e-02,  1.4251e-02,
         -3.3574e-02,  2.6518e-02, -4.1100e-02, -3.5351e-02,  4.1666e-02,
          3.2555e-02,  1.9023e-03, -3.2874e-02,  5.3508e-03, -1.6104e-02,
          2.0862e-02,  4.0813e-02, -8.4683e-03,  1.7769e-02, -1.7465e-02,
         -1.0925e-04,  6.1013e-03, -2.1344e-03, -2.1584e-02,  7.4184e-05

/usr/local/lib/python3.9/dist-packages/modeci_mdf/interfaces/pytorch/importer.py:517: FutureWarning: 'torch.onnx.symbolic_helper._set_opset_version' is deprecated in version 1.13 and will be removed in version 1.14. Please remove its usage and avoid setting internal variables directly.
  _set_opset_version(modeci_onnx_opset_version)
/usr/local/lib/python3.9/dist-packages/modeci_mdf/interfaces/pytorch/importer.py:526: FutureWarning: 'torch.onnx.symbolic_helper._set_opset_version' is deprecated in version 1.13 and will be removed in version 1.14. Please remove its usage and avoid setting internal variables directly.
  _set_opset_version(previous_opset_version)
/usr/local/lib/python3.9/dist-packages/modeci_mdf/interfaces/pytorch/importer.py:168: FutureWarning: 'torch.onnx._patch_torch._node_getitem' is deprecated in version 1.13 and will be removed in version 1.14. Please Internally use '_node_get' in symbolic_helper instead..
  {aname: convert_to_serializable(node[aname]) for aname in no


Init graph: ResNetGraph
Evaluating graph: ResNetGraph, root nodes: ['Conv_192'], with array format numpy
Evaluated the graph in PyTorch, output: [[-0.0287744 ...  0.0151076]] (NP (1, 1000) float64)
Passed all comparison tests!
Exported to MDF and ONNX
